In [12]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
import gradio as gr
from pydantic import BaseModel
from typing import List
import asyncio
import time
load_dotenv(override=True)

True

In [3]:
agents = []

agentsGpt4oMini = {
    "model": "gpt-4o-mini",
    "agent": OpenAI()
}

agents.append(agentsGpt4oMini)

agentsGpt5Mini = {
    "model": "gpt-5-mini",
    "agent": OpenAI()
}

agents.append(agentsGpt5Mini)
google_api_key = os.getenv("GOOGLE_API_KEY")
agentsGemini25Flash = {
    "model": "gemini-2.5-flash",
    "agent": OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/" )
}
agents.append(agentsGemini25Flash)
        
agents

[{'model': 'gpt-4o-mini', 'agent': <openai.OpenAI at 0x1743f939370>},
 {'model': 'gpt-5-mini', 'agent': <openai.OpenAI at 0x1743f939610>},
 {'model': 'gemini-2.5-flash', 'agent': <openai.OpenAI at 0x1743f9393d0>}]

In [4]:
# Carregando recussos 
with open("recursos/every-money-readme.md", "r", encoding="utf-8") as f:
    everyMoneyReadme = f.read()

with open("recursos/criticidade.json", "r", encoding="utf-8") as f:
    criticidade = f.read()

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

class Risco(BaseModel):
    nome: str
    criticidade: str
    descricao: str
    dano_causado: str
    probabilidade: str
    impacto: str

class Riscos(BaseModel):
    riscos: List[Risco]

In [5]:
build_riscos_system_prompt = f"""Você é um especialista em análise de riscos e deve analisar o seguinte projeto: {everyMoneyReadme}

Classifique os riscos de acordo com a criticidade: {criticidade}

Apresente pelo menos 3 riscos, classificados conforme a criticidade.

Não seja muito verboso na descrição dos riscos, sendo sucinto e direto.

Deve se atentar para as funcionalidades do projeto, e não apenas as tecnologias utilizadas.

O resultado deve ser apresentado em formato JSON, seguindo o modelo abaixo:
{{
    "Riscos": [
        {{
            "nome": "Nome do risco",
            "criticidade": "Criticidade do risco",
            "descricao": "Descrição do risco",
            "dano_causado": "Dano causado pelo risco",
            "probabilidade": "Probabilidade de ocorrência do risco",
            "impacto": "Impacto do risco"
        }}
    ]
}}"""

In [6]:
agentsGpt5Nano = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

evaluator_system_prompt = "Você é um avaliador de riscos e deve avaliar os riscos apresentados pelo analista de riscos."\
    "Avalie se os riscos estão de acordo com o projeto e se são relevantes."\
    "Se houver riscos que não estão de acordo com a descrição, avalie se eles são relevantes para o projeto."\
    "Se houver riscos que não estão de acordo com o dano causado, avalie se eles são relevantes para o projeto."\
    "Se houver riscos que não estão de acordo com o impacto, avalie se eles são relevantes para o projeto."\
    "Ao final sua resposta deve ser o risco isacceptable e o feedback para o analista de riscos."

def evaluator_user_prompt(reply, riscos, history):
    user_prompt = f"Temos aqui o histórico de avalição do risco: \n\n{history}\n\n"
    user_prompt += f"Aqui temos o risco apresentado pelo analista de riscos: \n\n{riscos}\n\n"
    user_prompt += f"Aqui temos a resposta do avaliador: \n\n{reply}\n\n"
    user_prompt += "Por favor avalie a resposta, respondendo se é aceitável e seu feedback. seguindo o modelo abaixo: \n\n"
    user_prompt += "{{ \n\n"
    user_prompt += "    \"is_acceptable\": true, \n\n"
    user_prompt += "    \"feedback\": \"Feedback do avaliador\" \n\n"
    user_prompt += "}}"
    return user_prompt


async def evaluate(reply, riscos, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, riscos, history)}]
    response = agentsGpt5Nano.beta.chat.completions.parse(model="gpt-5-nano", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [7]:
async def acionar_analista_de_riscos(message, agent) -> Riscos:
    model0 = agent['model']
    print(f"Gerando riscos com o modelo {model0}")
    agentModel = agent['agent']
    messages = [{"role": "system", "content": build_riscos_system_prompt}] + message
    response = agentModel.chat.completions.parse(model = model0, messages=messages, response_format=Riscos)
    return response.choices[0].message.parsed


In [20]:
NUMERO_MAXIMO_DE_REAVALIACOES = 1
async def gerar_riscos(agent) -> Riscos:
    try:
        riscos_criados = []
        messages = [{"role": "user", "content": "Gerar riscos para o projeto EveryMoney"}]
        riscos = await acionar_analista_de_riscos(messages, agent)
        valor_corrente = 0
        resposta_avaliador = ""
        while valor_corrente < NUMERO_MAXIMO_DE_REAVALIACOES:
            evaluation = await evaluate(resposta_avaliador, riscos, messages)
            if evaluation.is_acceptable:
                print("Passed evaluation - returning reply")
                riscos_criados.append(riscos)
            else:
                print("Failed evaluation - retrying")
                print(evaluation.feedback)
                resposta_avaliador += evaluation.feedback

            valor_corrente += 1
        return riscos_criados
    except Exception as e:
        print(f"Erro ao gerar riscos com o agente {agent['model']}: {e}")


def serializador_customizado(obj):
    if hasattr(obj, '__dict__'):
        return obj.__dict__
    return str(obj)     


In [19]:
async def main():
    print("--- Iniciando Promise.all (asyncio.gather) ---")
    inicio = time.perf_counter()
    execucoes_agentes = []
    for agent in agents:
        execucoes_agentes.append(gerar_riscos(agent))

    resultados = await asyncio.gather(*execucoes_agentes)
    print(json.dumps(resultados, indent=4, ensure_ascii=False, default=serializador_customizado))
    fim = time.perf_counter()
    print(f"Tempo total de execução: {fim - inicio:.2f} segundos")


In [21]:

await main()

--- Iniciando Promise.all (asyncio.gather) ---
Gerando riscos com o modelo gpt-4o-mini
Passed evaluation - returning reply
Gerando riscos com o modelo gpt-5-mini
Passed evaluation - returning reply
Gerando riscos com o modelo gemini-2.5-flash
Passed evaluation - returning reply
[
    [
        {
            "riscos": [
                {
                    "nome": "Exposição de Dados Sensíveis",
                    "criticidade": "Alta",
                    "descricao": "Possibilidade de vazamento de dados sensíveis dos usuários, como senhas e informações financeiras, devido a falhas de segurança.",
                    "dano_causado": "Roubo de identidade e comprometimento financeiro dos usuários.",
                    "probabilidade": "Média",
                    "impacto": "Alto"
                },
                {
                    "nome": "Erros de Cálculo nos Saldos",
                    "criticidade": "Média",
                    "descricao": "Possibilidade de erros nos método